In [11]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 6.5 MB/s eta 0:00:00


In [4]:
# Importing Libraries
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [5]:
# Reading the file and splitting them
pdf_reader = PyPDFLoader("/content/Power BI Ebook.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,)
chunks = text_splitter.split_documents(documents)

In [6]:
# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

<ipython-input-6-3e7171ca595d>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
# Calling the model
from langchain_xai import ChatXAI

llm = ChatXAI(
    model='grok-3-latest',
    temperature=0,
    max_tokens=None,
    api_key='YOUR_OPENAI_API_KEY'
)

In [8]:
#llm.invoke('Who is Roger Federer?')

In [9]:
# sets up a conversational retrieval chain using LangChain
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow up Input: {question}
Standalon question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False
                                           )

In [10]:
chat_history=[]
query = """?What are the components of Power BI?"""
result = qa({"question": query, "chat_history": chat_history})
print(result['answer'])

<ipython-input-10-827c7d2d03c9>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": query, "chat_history": chat_history})


The components of Power BI, based on the provided context, include the following key elements and features:

1. **Power BI Desktop**: A Windows application that is part of the Power BI suite, enabling users to create interactive data visualizations, reports, and dashboards. It offers comprehensive features for:
   - **Data Modeling**: Building powerful models and flexible measures, including automatic model creation when connecting to data.
   - **Data Preparation**: Using tools like Power Query Editor to connect, clean, and mashup data from over 80 data sources (both on-premises and cloud), shape and transform data, and extend with custom data connectors.
   - **Data Analysis**: Utilizing a high-performance, in-memory engine for analysis.
   - **Data Visualization**: Creating interactive visualizations and dashboards.

2. **Power Query Editor**: A tool within Power BI for data preparation, allowing users to connect to various data sources, clean, and transform data for analysis.

3. *

## **RAG Evaluation**

In [11]:
sample_queries = [
    {
        "question": "What are the main components of Power BI?",
        "answer": "The main components of Power BI are Power BI Desktop, Power BI Service, and Power BI Mobile.",
        "reference": "Power BI has three main components: Desktop (Windows app for creating reports), Service (cloud platform for sharing and collaboration), and Mobile (app for viewing reports on mobile devices)."
    },
    {
        "question": "What are the key features of Power BI Desktop?",
        "answer": "Key features include data connection and transformation, data modeling, interactive visualizations, report publishing, and collaboration tools.",
        "reference": "Power BI Desktop allows users to get data, analyze it with DAX, visualize using 150+ visuals, publish to cloud or on-premises, and collaborate with team members."
    },
    {
        "question": "How does Power BI integrate with Excel?",
        "answer": "Power BI can analyze data in Excel, import Excel data, connect to Excel workbooks, and pin Excel ranges to dashboards.",
        "reference": "Power BI integrates with Excel by enabling analysis in Excel, data import, connection to workbooks, and uploading Excel files for dashboard pinning."
    },
    {
        "question": "What is the Power Query Editor used for?",
        "answer": "Power Query Editor is used for data transformation and cleansing before importing data into models or visualizations.",
        "reference": "Power Query Editor includes tools like ribbon tabs, data preview, query settings, and footer with data stats for transforming and shaping data."
    },
    {
        "question": "How can you group data in Power BI?",
        "answer": "You can group data using the 'Group By' feature in the Transform tab of the Query Editor by selecting columns and specifying aggregate calculations.",
        "reference": "In Power BI's Query Editor, the Group By dialog allows users to select columns and apply aggregate functions to group data."
    }
]


In [12]:
import pandas as pd
df = pd.DataFrame(sample_queries)
df.head(5)

,question,answer,reference
0,What are the main components of Power BI?,The main components of Power BI are Power BI D...,Power BI has three main components: Desktop (W...
1,What are the key features of Power BI Desktop?,Key features include data connection and trans...,"Power BI Desktop allows users to get data, ana..."
2,How does Power BI integrate with Excel?,"Power BI can analyze data in Excel, import Exc...",Power BI integrates with Excel by enabling ana...
3,What is the Power Query Editor used for?,Power Query Editor is used for data transforma...,Power Query Editor includes tools like ribbon ...
4,How can you group data in Power BI?,You can group data using the 'Group By' featur...,"In Power BI's Query Editor, the Group By dialo..."


In [13]:
retriever = db.as_retriever()

In [14]:
def process_query(query):
  chat_history = []
  result = qa({"question": query, "chat_history": chat_history})
  relevant_docs = retriever.invoke(query)
  print(result['answer'])
  return result['answer'], relevant_docs

In [15]:
process_query("What is the Power Query Editor used for?")

The Power Query Editor is used for data transformation and cleansing. It is a tool built into Power BI, Excel, and other Microsoft products, providing users with a powerful set of tools to transform and shape their data before it is imported into a data model or visualization.


('The Power Query Editor is used for data transformation and cleansing. It is a tool built into Power BI, Excel, and other Microsoft products, providing users with a powerful set of tools to transform and shape their data before it is imported into a data model or visualization.',
 [Document(id='05a59dc3-7752-436a-9c2c-94695befea67', metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2024-03-16T19:22:04+08:00', 'author': 'Satyajit', 'moddate': '2024-03-16T19:22:04+08:00', 'source': '/content/Power BI Ebook.pdf', 'total_pages': 43, 'page': 7, 'page_label': '8'}, page_content='© zepanalytics.com\n8\nDATA ANALYTICS | Power BI\n● Power Query Editor is a data transformation and cleansing tool that is built into Power BI, \nExcel, and other Microsoft products. \n● It provides users with a powerful set of tools for transforming and shaping their data \nbefore it is imported into a data model or visualization.\nUser Interface:\n1. Ri

#### **RAGAS Framework**

In [16]:
pip install ragas

In [17]:
results = []

for _, row in df.iterrows():
  question = row['question']
  ground_truth = row['answer']

  answer, relevant_docs = process_query(question)

  results.append({
      "user_input": question,
      "reference": ground_truth,
      "response": answer,
      "retrieved_contexts": [relevant_docs[0].page_content]
  })

The main components of Power BI, based on the provided context, include the following:

1. **Power BI Desktop**: A Windows application that enables users to create interactive data visualizations, reports, and dashboards. It offers features for data modeling, preparation, analysis, and visualization. Key functionalities include:
   - **Get Data**: Connect to over 80 data sources (both on-premises and cloud), shape, transform, and clean data for analysis, and extend with custom data connectors.
   - **Analyze**: Build powerful data models and flexible measures with an in-memory engine for high performance and automatic model creation when connecting to data.

2. **Power Query Editor**: A tool within Power BI for data preparation, allowing users to connect, clean, and transform data from various sources for analysis.

3. **Data Analysis Expression (DAX)**: A formula language used in Power BI for creating custom calculations and measures to enhance data analysis.

4. **Power BI Relationsh

In [18]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

In [19]:
evaluation_dataset = EvaluationDataset.from_list(results)

In [20]:
evaluator_llm = LangchainLLMWrapper(llm)

In [21]:
ragas_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm = evaluator_llm
)

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
#ragas_result (grok-3-mini)

In [23]:
ragas_result

{'context_recall': 0.7100, 'faithfulness': 0.6321, 'factual_correctness(mode=f1)': 0.5760}

#### **grok-3-mini results:**

To interpret the RAGAS results from your evaluation of the RAG-based pipeline, let's break down the metrics:

- Context Recall (0.5500): This metric indicates that the model correctly recalls relevant information from the provided context about 55% of the time. While it's above 50%, there’s room for improvement, especially as higher recall values are generally preferred for effective context utilization.

- Faithfulness (0.4407): This score suggests that approximately 44% of the responses are consistent with the source context or information. A faithfulness score below 50% indicates that the model may generate responses that diverge from the original context, which can lead to misinformation.

- Factual Correctness (F1 Score of 0.3980): The F1 score combines precision and recall for factual accuracy, and a score of about 40% indicates that less than half of the generated answers are factually correct. This is relatively low and highlights a significant area for improvement.

#### **grok-3-latest results**

- Context Recall (0.7500): This score indicates that the model now correctly recalls relevant context information 75% of the time. This is a strong improvement and suggests that the model is effectively utilizing the context provided.

- Faithfulness (0.6420): With a faithfulness score of 64.2%, the model is generating responses that are consistent with the source context more than 60% of the time. This is a notable improvement, indicating greater reliability in the responses.

- Factual Correctness (F1 Score of 0.5940): An F1 score of approximately 59.4% suggests that the model is now producing factually correct answers nearly 60% of the time. This is a significant enhancement and indicates that the model is becoming more accurate.

#### **n-Gram Metrics**

- BLEU
- ROUGE

In [24]:
pip install rouge

In [25]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

In [26]:
rouge = Rouge()
bleu = []
rouge_one = []

In [27]:
for item in results:
  reference = [item["reference"].split()]
  hypothesis = item["response"].split()
  bleu.append(sentence_bleu(reference, hypothesis))

  scores = rouge.get_scores(item["response"], item["reference"])[0]
  rouge_one.append(scores["rouge-1"]["f"])

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [28]:
bleu

[0.018896194518248415,
 2.0053429233337484e-155,
 2.728467787769403e-155,
 0.1748812694918866,
 9.295075002399697e-79]

In [29]:
rouge_one

[0.07650273111051392,
 0.12068965304994059,
 0.22222221973847617,
 0.4814814773113856,
 0.2300884924395019]

In [ ]:
#df.to_csv('queries.csv')

#### **Model-based metrics**

- BARTScore
- BERTScore

In [45]:
from transformers import BartForConditionalGeneration, BartTokenizer
from bert_score import BERTScorer
import torch

In [46]:
# Initialize the BART model and tokenizer
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Initialize the BERTScorer
bert_scorer = BERTScorer(lang='en',rescale_with_baseline=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
bert_scores = []
bart_scores = []

for item in results:
  inputs = bart_tokenizer(item["response"], return_tensors='pt', truncation=True, padding=True )
  with torch.no_grad():
    bart_score = bart_model(**inputs).logits
  bart_scores.append(bart_score.mean().item())

  P, R, F1 = bert_scorer.score([item["response"]], [item["reference"]])
  bert_scores.append(F1.numpy().mean())

In [52]:
bart_scores

[0.058085232973098755,
 0.05085964873433113,
 0.03933323174715042,
 -0.01700144074857235,
 0.05467526614665985]

In [53]:
bert_scores

[np.float32(0.044678167),
 np.float32(0.07860397),
 np.float32(0.22398238),
 np.float32(0.5322963),
 np.float32(0.29302132)]